In [ ]:
# Install OSMNX only if working on Google Colab
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    !pip install osmnx

In [1]:
# Load datasets if working on Google Colab
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials

    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)

    file_id = '...'
    downloaded = drive.CreateFile({'id':file_id})
    downloaded.FetchMetadata(fetch_all=True)
    downloaded.GetContentFile(downloaded.metadata['title'])

    f = open("V2data_6mounts2022.csv.zip", "wb")
    f.write(downloaded.content.getbuffer())
    f.close()

    !unzip V2data_6mounts2022.csv.zip

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import urllib.request
import json 
import osmnx as ox
sns.set_style('whitegrid')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 300)

## Dataset Import

In [3]:
if IN_COLAB: df_main_url = '/content/data.csv'
else: df_main_url = '../../Information/output/datav2.csv'
df_main = pd.read_csv(df_main_url, dtype = {'place.id': object})

### Drop unneeded columns and rename columns to naming convention used

In [4]:
not_needed_columns = ['value.Vehicle_Mode', 'speed', '_id',]
df_main.drop(not_needed_columns, axis=1, inplace=True, errors='ignore')
df_main.rename(columns = {'suitcase.id':'suitcase_id', 'date.utc':'date_utc', 
                    'edge.id':'edge_id', 'edge.osmid':'edge_osmid', 
                    'place.id':'place_id', 'osm.highway':'osm_highway'}, inplace=True, errors='ignore')

### Make a copy of the dataframe to work with

In [5]:
df = df_main.copy()
# df_test = df[~df['place.id'].isna()]

## Data Cleaning

In [6]:
litter_columns = []
for item in df.columns.to_list():
    if item.isdigit(): litter_columns.append(item) 

def clean_df(df_arg, mapping):
    df = df_arg
    df = df.dropna(subset=[mapping]).copy()
    df.drop('place_id', axis=1, inplace=True)
    df['date_utc'] = pd.to_datetime(df['date_utc']).dt.date
    df[litter_columns] = df[litter_columns].fillna(0)
    df[litter_columns] = df[litter_columns].astype(np.int64) 
    df['total_litter'] = df[litter_columns].sum(axis=1)
    return df

df = clean_df(df, 'edge_id')

## Data Aggregation

In [7]:
aggregation_type = 'sum'
to_agg = {'edge_osmid' : 'first', 'osm_highway' : 'first', 'total_litter' : aggregation_type}
for litter in litter_columns:
    to_agg[litter] = aggregation_type
df = df.groupby(['date_utc', 'edge_id'], as_index=False).agg(to_agg)

## Darkzone Dataframe Creation

### Dataframe preparation

In [8]:
df.drop('total_litter', axis=1, inplace=True, errors='ignore')
df.drop(litter_columns, axis=1, inplace=True, errors='ignore')
df['date_utc'] = df['date_utc'].astype(str)

### Darzones as Missing Edges Dictionary

In [9]:
edges_dictionary = df['edge_id'].unique()

# Make a dictionary with the date as key and the edges existing within that day in a list as their value
today = ''
existing_edges = {}
for index, row in df.iterrows():
    if row['date_utc'] != today:
        today = row['date_utc']
        existing_edges[today] = []
    if row['edge_id'] not in existing_edges[today]:
        existing_edges[today].append(row['edge_id'])

# Compare the existing edges in a day with the list of all the unique edges, 
#   add the edges not in a day in a dictionary, 
#   where the key is the date and the value is a list of the missing edges in that dataframe
missing_edges = {}
for key, value in existing_edges.items():
    missing_edges[key] = list(set(edges_dictionary).difference(value))
missing_edges_list = []
for key, value in missing_edges.items():
    for v in value:
        missing_edges_list.append([key, v])

### Create Darkzones Pandas Dataframe

In [10]:
df_darkzones = pd.DataFrame(list(missing_edges_list), columns = ['date_utc', 'edge_id']) 
edges_dictionary_extended = df.groupby(['edge_id'], as_index=False).agg({'edge_osmid':'first', 'osm_highway':'first'})
df_darkzones = pd.merge(df_darkzones, edges_dictionary_extended, how='outer', on='edge_id')
df_darkzones['date_utc'] = pd.to_datetime(df_darkzones['date_utc'], format='%Y-%m-%d')
df_darkzones = df_darkzones.sort_values(by="date_utc")
df_darkzones = df_darkzones.reset_index(drop=True)

In [11]:
df = df_darkzones.copy()
df['row_type'] = "darkzone"
df['edge_osmid'] = df['edge_osmid'].astype(int)

## Feauture Creation

### Splitting the date

In [12]:
df['Year'] = pd.DatetimeIndex(df['date_utc']).year.astype(object)
df['month'] = pd.DatetimeIndex(df['date_utc']).month.astype(object)
df['day'] = pd.DatetimeIndex(df['date_utc']).day.astype(object)

### Add weekday

In [13]:
weekdays = ('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday')
df['weekday'] = pd.to_datetime(df['date_utc']).dt.date.apply(lambda x: x.weekday())
df['weekday'] = df['weekday'].apply(lambda x: weekdays[x])

### Add holiday

In [14]:
from datetime import datetime
from datetime import timedelta
from holidays import Switzerland

holiday = [holiday for holiday in Switzerland(years=[2021, 2022]).items()]
for day in set(holiday):
    holiday.append(((day[0] + timedelta(days=1)), day[1]))
    holiday.append(((day[0] + timedelta(days=2)), day[1]))

holidays_df = pd.DataFrame(holiday, columns=["date", "holiday"])
holidays_df['holiday'] = holidays_df['holiday'].astype(str)
df['holiday'] = df['date_utc'].apply(lambda x: 1 if x in holidays_df['date'].values else 0)

/var/folders/18/k719ypc560z2_2smxlh0k35c0000gn/T/ipykernel_1548/4227541218.py:12: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  df['holiday'] = df['date_utc'].apply(lambda x: 1 if x in holidays_df['date'].values else 0)


### Add latitude and longitude

#### Make a dataframe with edge_id and it's latitudes based on geojson file

In [15]:
add_coordinates = True
if add_coordinates:
    if IN_COLAB:
        with urllib.request.urlopen('https://raw.githubusercontent.com/dominik117/cortexia-darkzones-prediction/main/src/data/edges.geojson') as url:
            data = json.loads(url.read().decode())
    else:
        with open('../references/edges.geojson') as f:
            data = json.load(f)
 
    df_edges = pd.DataFrame(data['features'])  # <-- The only column needed
    df_edges = pd.concat([df_edges.drop(['properties'], axis=1), df_edges['properties'].apply(pd.Series)], axis=1)  # <--Explode properties dictionary inside cells
    df_edges.rename(columns = {'id':'edge_id'}, inplace = True)
    # Rearrange BBOX (lat_north, lat_south, lon_east, lon_west)
    def sort_bbox(x):
        lat = sorted([x[1], x[3]], key=float, reverse=True)
        lon = sorted([x[0], x[2]], key=float, reverse=True)
        return lat + lon
    df_edges['bbox'] = df_edges['bbox'].apply(sort_bbox)
    df_coordinates = df_edges[['edge_id', 'bbox']].copy()
    bbox_exploded = pd.DataFrame(df_coordinates["bbox"].to_list(), columns=['lat_north', 'lat_south', 'lon_east', 'lon_west'])
    df_coordinates = pd.concat([df_coordinates, bbox_exploded], axis=1)
    df_coordinates.drop(['bbox'], axis=1, inplace=True)
    df = pd.merge(df, df_coordinates, how="left", on="edge_id")
else:
    print("To add coordinates change the add_coordinates variable to true")

### Add length of edge (street segment)

#### Formula to get distance between two points on Earth

In [16]:
from math import atan, cos, radians, sin, tan, asin, sqrt

def haversine_distance(lat1, lon1, lat2, lon2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    EQUATORIAL_RADIUS = 6378 # Radius of earth in kilometers
    return c * EQUATORIAL_RADIUS

def lamberts_ellipsoidal_distance(lat1: float, lon1: float, lat2: float, lon2: float) -> float:
    AXIS_A = 6378137.0
    AXIS_B = 6356752.314245
    EQUATORIAL_RADIUS = 6378137
    flattening = (AXIS_A - AXIS_B) / AXIS_A
    b_lat1 = atan((1 - flattening) * tan(radians(lat1)))
    b_lat2 = atan((1 - flattening) * tan(radians(lat2)))
    sigma = haversine_distance(lat1, lon1, lat2, lon2) / EQUATORIAL_RADIUS
    P_value = (b_lat1 + b_lat2) / 2
    Q_value = (b_lat2 - b_lat1) / 2
    X_numerator = (sin(P_value) ** 2) * (cos(Q_value) ** 2)
    X_demonimator = cos(sigma / 2) ** 2
    X_value = (sigma - sin(sigma)) * (X_numerator / X_demonimator)
    Y_numerator = (cos(P_value) ** 2) * (sin(Q_value) ** 2)
    Y_denominator = sin(sigma / 2) ** 2
    Y_value = (sigma + sin(sigma)) * (Y_numerator / Y_denominator)
    distance = abs(EQUATORIAL_RADIUS * (sigma - ((flattening / 2) * (X_value + Y_value))))
    return int(distance)

In [17]:
df['edge_length'] = df.apply(lambda x: lamberts_ellipsoidal_distance(x.lat_north, x.lon_east, x.lat_south, x.lon_west), axis=1)

### Add weather

In [18]:
if IN_COLAB: df_weather = pd.read_csv("https://raw.githubusercontent.com/dominik117/cortexia-darkzones-prediction/main/src/data/weather_basel_2021-2022.csv")
else: df_weather = pd.read_csv("../src/data/weather_basel_2021-2022.csv")

# Data obtained from https://www.meteoblue.com/en/weather/archive/export/basel_switzerland_2661604
df['date_utc'] = pd.to_datetime(df['date_utc']).dt.date
weather_columns = {'location':'date_utc', 'Basel':'temperature_max', 'Basel.1':'temperature_min', 'Basel.2':'temperature_mean', 'Basel.3':'precipitation',
                   'Basel.4':'snowfall', 'Basel.5':'humidity_max', 'Basel.6':'humidity_min', 'Basel.7':'humidity_mean', 'Basel.8':'cloud_coverage',
                   'Basel.9':'wind_speed_max', 'Basel.10':'wind_speed_min', 'Basel.11':'wind_speed_mean'}
df_weather.rename(columns = weather_columns, inplace = True)
df_weather.drop(['Basel.12'], axis=1, inplace=True)
df_weather = df_weather.iloc[9:].copy()  # <-- Rows with metadata
df_weather['date_utc'] = pd.to_datetime(df_weather['date_utc']).dt.date  # <-- Match main df type and name
weather_date = df_weather.pop('date_utc')  # <-- Pop date so it doesn't get converted
df_weather = df_weather.apply(pd.to_numeric)
df_weather = df_weather.round(decimals = 1)
df_weather.insert(0, 'date_utc', weather_date) # <-- Reassign unmodified date to df
df = pd.merge(df, df_weather, how="left", on="date_utc")
df['date_utc'] = df['date_utc'].astype(str)

### Add features from Open Street Maps (OSM)

#### Make the datasets with points of interest

In [19]:
add_points_of_interest = True
# Since we need longitude and latitude to add OSM features, turn it on when adding points of interest if off
if add_coordinates is False and add_points_of_interest:
    add_points_of_interest = True
    add_coordinates = True
    print("WARNING: add_coordinates was turned on since it's needed to aggregate points of interest")
    print(f"add_coordinates = {add_coordinates},  add_points_of_interest = {add_points_of_interest}")

# Extraction of the features on the dictionary from OSM
basel = 'Basel, Basel, Switzerland'
# Features dictionary at https://wiki.openstreetmap.org/wiki/Map_features

tags = {'amenity': ['vending_machine', 'bench', 'bar', 'fast_food', 'ice_cream', 'kindergarten', 'school', 'hospital', 'cinema', 
                    'fountain', 'dog_toilet', 'recycling', 'waste_basket', 'waste_disposal', 'childcare', 'marketplace',
                    'bus_station', 'fuel', 'taxi', 'parking', 'atm', 'clinic', 'nightclub', 'toilets'
                    ]}
amenity = ox.geometries_from_place(basel, tags=tags)
tags = {'leisure': 'park'}
leisure = ox.geometries_from_place(basel, tags=tags)

/Users/bacher/opt/anaconda3/envs/cortexia/lib/python3.10/site-packages/osmnx/geometries.py:805: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for merged_outer_linestring in list(merged_outer_linestrings):
/Users/bacher/opt/anaconda3/envs/cortexia/lib/python3.10/site-packages/osmnx/geometries.py:805: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  for merged_outer_linestring in list(merged_outer_linestrings):
/Users/bacher/opt/anaconda3/envs/cortexia/lib/python3.10/site-packages/osmnx/geometries.py:805: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts 

#### Make dataframe with points of interest and their coordinates

In [20]:
df_osm = pd.DataFrame(amenity)  # <-- Convert to DF
df_osm = df_osm[['amenity', 'geometry']].copy()  # <-- Select the only needed columns
df_osm['osm_id'] = df_osm.index.to_numpy()  # <-- Detach the index and assign it to a normal column
df_osm.reset_index(drop=True, inplace=True)  # <-- Drop index
osm_id_exploded = pd.DataFrame(df_osm["osm_id"].to_list(), columns=['type', 'osm'])  # <-- Explode index since it contains two indices
df_osm = pd.concat([df_osm, osm_id_exploded], axis=1) 
df_osm.drop(['osm_id', 'osm'], axis=1, inplace=True)
df_osm = df_osm[df_osm['type'] == 'node']  # <-- Drop Multipoligon points
#### Clean the coordinates from the GeoPandas geometry format to latitude and longitude columns
df_osm['lon'] = df_osm[df_osm['type'] == "node"]['geometry'].apply(lambda p: p.x)
df_osm['lat'] = df_osm[df_osm['type'] == "node"]['geometry'].apply(lambda p: p.y)
df_osm.drop(['geometry', 'type'], axis=1, inplace=True)

#### Merge the amenity based on coordinates with the corresponding coordinates from the edge_id

In [21]:
if add_points_of_interest:
    df_edges_coordinates = df[['edge_id', 'lat_north', 'lat_south', 'lon_east', 'lon_west']].copy()
    df_edges_coordinates = df_edges_coordinates.drop_duplicates(subset='edge_id', keep='first')

    # Make a list of the edges that have an amenity to them based on lat,lon conditional
    def is_between(a, x, b):
        return min(a, b) < x < max(a, b)
    edges_dict = []
    for edges_row in df_edges_coordinates.itertuples():
        for osm_row in df_osm.itertuples():
            if is_between(edges_row.lat_south, osm_row.lat, edges_row.lat_north) and is_between(edges_row.lon_west, osm_row.lon, edges_row.lon_east):
                edges_dict.append([edges_row.edge_id, osm_row.amenity])

    # Group by edge_id and get the value counts per amenity
    df_edges_dict = pd.DataFrame(edges_dict, columns = ['edge_id', 'amenity'])
    df_edges_dict = df_edges_dict.groupby('edge_id')['amenity'].value_counts().unstack(fill_value=0).reset_index()
    df = pd.merge(df, df_edges_dict, how="left", on="edge_id")
    osm_columns = list(df_edges_dict.columns[1:])
    df[osm_columns] = df[osm_columns].fillna(value=0)  # <-- Fill missing values, since not all edges have amenities
    df[osm_columns] = df[osm_columns].astype(int)
else:
    print("To add a count of points of interest from OSM, change add_points_of_interest variable to True")

In [24]:
df

,date_utc,edge_id,edge_osmid,osm_highway,row_type,Year,month,day,weekday,holiday,lat_north,lat_south,lon_east,lon_west,edge_length,temperature_max,temperature_min,temperature_mean,precipitation,snowfall,humidity_max,humidity_min,humidity_mean,cloud_coverage,wind_speed_max,wind_speed_min,wind_speed_mean,atm,bar,bench,childcare,cinema,clinic,fast_food,fountain,fuel,ice_cream,kindergarten,marketplace,nightclub,parking,recycling,school,taxi,toilets,vending_machine,waste_basket,waste_disposal
0,2022-01-02,"(1410128103, 3832624747, 0)",601226185,service,darkzone,2022,1,2,Sunday,1,47.571150,47.570980,7.580794,7.580724,27,16.3,6.4,10.2,0.0,0.0,73.0,44.0,61.8,41.6,17.3,4.0,9.7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2022-01-02,"(3840022060, 252822620, 0)",23348435,residential,darkzone,2022,1,2,Sunday,1,47.581258,47.580275,7.591719,7.591177,156,16.3,6.4,10.2,0.0,0.0,73.0,44.0,61.8,41.6,17.3,4.0,9.7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2022-01-02,"(8740263820, 8740263891, 0)",944003107,footway,darkzone,2022,1,2,Sunday,1,47.559394,47.559373,7.582399,7.582348,1,16.3,6.4,10.2,0.0,0.0,73.0,44.0,61.8,41.6,17.3,4.0,9.7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2022-01-02,"(1386646225, 3798166958, 0)",376382351,footway,darkzone,2022,1,2,Sunday,1,47.546516,47.546487,7.588681,7.588575,1,16.3,6.4,10.2,0.0,0.0,73.0,44.0,61.8,41.6,17.3,4.0,9.7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2022-01-02,"(3830452887, 3830452889, 0)",379717822,footway,darkzone,2022,1,2,Sunday,1,47.553356,47.553290,7.586263,7.586233,10,16.3,6.4,10.2,0.0,0.0,73.0,44.0,61.8,41.6,17.3,4.0,9.7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1782718,2022-06-30,"(3832502175, 3832502177, 0)",379898672,footway,darkzone,2022,6,30,Thursday,0,47.560138,47.560082,7.612677,7.612504,4,31.2,14.8,22.8,14.2,0.0,88.0,35.0,64.8,26.1,28.8,0.8,13.5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1782719,2022-06-30,"(7239920874, 206101871, 0)",601210265,secondary,darkzone,2022,6,30,Thursday,0,47.557127,47.557121,7.564945,7.564796,0,31.2,14.8,22.8,14.2,0.0,88.0,35.0,64.8,26.1,28.8,0.8,13.5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1782720,2022-06-30,"(6780195944, 7981706049, 0)",855964114,path,darkzone,2022,6,30,Thursday,0,47.559417,47.559389,7.609174,7.609105,2,31.2,14.8,22.8,14.2,0.0,88.0,35.0,64.8,26.1,28.8,0.8,13.5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1782721,2022-06-30,"(6704338879, 6702684680, 0)",713181999,footway,darkzone,2022,6,30,Thursday,0,47.557450,47.557381,7.573679,7.572956,1,31.2,14.8,22.8,14.2,0.0,88.0,35.0,64.8,26.1,28.8,0.8,13.5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
